In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [74]:
import warnings
warnings.filterwarnings("ignore")

In [75]:
data = pd.read_csv('facies_vectors_0.csv')
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS','GR_diff_up', 'ILD_log10_diff_up', 'DeltaPHI_diff_up', 'PHIND_diff_up', 'PE_diff_up', 'NM_M_diff_up', 'RELPOS_diff_up']
feature_names_23 = ['GR_diff_up', 'ILD_log10_diff_up', 'DeltaPHI_diff_up', 'PHIND_diff_up', 'PE_diff_up', 'NM_M_diff_up', 'RELPOS_diff_up','GR_diff_down', 'ILD_log10_diff_down', 'DeltaPHI_diff_down', 'PHIND_diff_down', 'PE_diff_down', 'NM_M_diff_down', 'RELPOS_diff_down','GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
feature_names_original = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS','GR_diff_up']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']
data = data.fillna(data['PE'].mean())

In [76]:
def find_diff(row, well):
    if len(prev_depth_features[well]) == 0:
        prev_depth_features[well] = row.values[4:]
        return
    diff = row.values[4:] - prev_depth_features[well]
    prev_depth_features[well] = row.values[4:]
    return diff
data_well = dict()
data_well_inverse = dict()
prev_depth_features = dict()
new_data = pd.DataFrame()
prev_class= dict()
data_save = pd.DataFrame()
for well in set(data['Well Name']):
    prev_depth_features[well] = []
    prev_class[well] = []
    data_well[well] = data[data['Well Name'] == well]
    data_well[well] = data_well[well].sort_values(by=['Depth'])
    data_save = data_well[well].iloc[::-1]
    data_well[well]['diff_up'] = data_well[well].apply(lambda row: find_diff(row, well), axis=1)
    prev_depth_features[well] = []
    prev_class[well] = []

    data_well[well] = data_well[well].dropna()
    data_well[well]['GR_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][0], axis=1)
    data_well[well]['ILD_log10_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][1], axis=1)
    data_well[well]['DeltaPHI_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][2], axis=1)
    data_well[well]['PHIND_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][3], axis=1)
    data_well[well]['PE_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][4], axis=1)
    data_well[well]['NM_M_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][5], axis=1)
    data_well[well]['RELPOS_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][6], axis=1)

    new_data = pd.concat([new_data, data_well[well]])
    new_data = new_data.drop(['diff_up'], axis=1)

In [77]:
def find_diff_all23(row, well):
    if len(prev_depth_features_all23[well]) == 0:
        prev_depth_features_all23[well] = row.values[4:]
        return
    diff = row.values[4:] - prev_depth_features_all23[well]
    prev_depth_features_all23[well] = row.values[4:]
    return diff
data_well_all23 = dict()
data_well_inverse_all23 = dict()
prev_depth_features_all23 = dict()
new_data_all23 = pd.DataFrame()
prev_class_all23= dict()
data_save_all23 = pd.DataFrame()
for well in set(data['Well Name']):
    prev_depth_features_all23[well] = []
    prev_class_all23[well] = []
    data_well_all23[well] = data[data['Well Name'] == well]
    data_well_all23[well] = data_well_all23[well].sort_values(by=['Depth'])
    data_save_all23 = data_well_all23[well].iloc[::-1]
    data_well_all23[well]['diff_up'] = data_well_all23[well].apply(lambda row: find_diff_all23(row, well), axis=1)
    prev_depth_features_all23[well] = []
    prev_class_all23[well] = []
    data_save_all23 = data_save_all23.apply(lambda row: find_diff_all23(row, well), axis=1)
    data_well_all23[well]['diff_down'] = data_save_all23.iloc[::-1]
    data_well_all23[well] = data_well_all23[well].dropna()
    data_well_all23[well]['GR_diff_up'] = data_well_all23[well].apply(lambda row: row['diff_up'][0], axis=1)
    data_well_all23[well]['ILD_log10_diff_up'] = data_well_all23[well].apply(lambda row: row['diff_up'][1], axis=1)
    data_well_all23[well]['DeltaPHI_diff_up'] = data_well_all23[well].apply(lambda row: row['diff_up'][2], axis=1)
    data_well_all23[well]['PHIND_diff_up'] = data_well_all23[well].apply(lambda row: row['diff_up'][3], axis=1)
    data_well_all23[well]['PE_diff_up'] = data_well_all23[well].apply(lambda row: row['diff_up'][4], axis=1)
    data_well_all23[well]['NM_M_diff_up'] = data_well_all23[well].apply(lambda row: row['diff_up'][5], axis=1)
    data_well_all23[well]['RELPOS_diff_up'] = data_well_all23[well].apply(lambda row: row['diff_up'][6], axis=1)
    data_well_all23[well]['GR_diff_down'] = data_well_all23[well].apply(lambda row: row['diff_down'][0], axis=1)
    data_well_all23[well]['ILD_log10_diff_down'] = data_well_all23[well].apply(lambda row: row['diff_down'][1], axis=1)
    data_well_all23[well]['DeltaPHI_diff_down'] = data_well_all23[well].apply(lambda row: row['diff_down'][2], axis=1)
    data_well_all23[well]['PHIND_diff_down'] = data_well_all23[well].apply(lambda row: row['diff_down'][3], axis=1)
    data_well_all23[well]['PE_diff_down'] = data_well_all23[well].apply(lambda row: row['diff_down'][4], axis=1)
    data_well_all23[well]['NM_M_diff_down'] = data_well_all23[well].apply(lambda row: row['diff_down'][5], axis=1)
    data_well_all23[well]['RELPOS_diff_down'] = data_well_all23[well].apply(lambda row: row['diff_down'][6], axis=1)
    data_well_all23[well] = data_well_all23[well].drop(['diff_up'], axis=1)
    data_well_all23[well] = data_well_all23[well].drop(['diff_down'], axis=1)
    new_data_all23 = pd.concat([new_data_all23, data_well_all23[well]])
#     new_data_all23 = new_data_all23.drop(['diff_up'], axis=1)
#     new_data_all23 = new_data_all23.drop(['diff_down'], axis=1)

In [78]:
def change_23(x):
    if (x == 2):
        return 0
    if (x == 3):
        return 1
    return x
new_data_2 = new_data_all23[new_data_all23['Facies'] == 2]
new_data_3 = new_data_all23[new_data_all23['Facies'] == 3]
new_data_23 = pd.concat([new_data_2, new_data_3])
new_data_23['Facies'] = new_data_23.apply(lambda row: change_23(row['Facies']), axis = 1)

In [79]:
def augment_features_window(X, N_neig):
    N_row = X.shape[0]
    N_feat = X.shape[1]
    X = np.vstack((np.zeros((N_neig, N_feat)),np.zeros((N_neig, N_feat)), X, np.zeros((N_neig, N_feat)),np.zeros((N_neig, N_feat))))
    X_aug = np.zeros((N_row, N_feat*(4*N_neig+1)))
    for r in np.arange(N_row) + N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
            if c != 0:
                this_row = np.hstack((this_row, (X[r] + X[r+c])/2))
        #print(len(this_row))
        X_aug[r-N_neig] = this_row

    return X_aug

def augment_features_gradient(X, depth):
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad

def augment_features(X, well, depth, N_neig=1):
    X_aug = np.zeros((X.shape[0], X.shape[1]*(4*N_neig+1)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        #print(X_aug_win)
        #X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        #print(X_aug_grad)
        X_aug[w_idx, :] = X_aug_win
        #X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
        
    return X_aug


In [80]:
import numpy as np
import xgboost as xgb
import math
from sklearn.preprocessing import OneHotEncoder

print('start running example to used customized objective function')

params_23 = {'max_depth': 2, 'eta': 0.1, 'silent': 1,
          'objective': 'multi:softprob', 'num_class': 2}
params = {'max_depth': 2, 'eta': 0.1, 'silent': 1,
          'objective': 'multi:softprob', 'num_class': 9}

num_round = 2
def my_softmax(preds, dtrain):
    labels = dtrain.get_label()
    labels_hot = OneHotEncoder(sparse=False, n_values=9).fit_transform(labels.reshape(-1, 1))
    grad = preds - labels_hot
    hess = preds * (1.0-preds)

    return grad.flatten(), hess.flatten()
def my_softmax_23(preds, dtrain):
    labels = dtrain.get_label()
    labels_hot = OneHotEncoder(sparse=False, n_values=2).fit_transform(labels.reshape(-1, 1))
    grad = preds - labels_hot
    hess = preds * (1.0-preds)

    return grad.flatten(), hess.flatten()

start running example to used customized objective function


In [81]:
def number_of_outlier(y_res):
    outliers = 0
    if y_res[0] != y_res[1]:
        outliers += 1
    if y_res[-1] != y_res[-2]:
        outliers += 1
    for index in range(1,len(y_res)-1):
        if ((y_res[index] != y_res[index-1]) and (y_res[index] != y_res[index+1])):
            outliers += 1
    return outliers/len(y_res)
            

In [82]:
def most_similar(y_res, y_test):
    for index in range(len(y_res)):
        if (y_res[index] != y_test[index]):
            classes[y_res[index], y_test[index]] += 1
            

In [83]:
new_data.tail(3)

,DeltaPHI,DeltaPHI_diff_up,Depth,Facies,Formation,GR,GR_diff_up,ILD_log10,ILD_log10_diff_up,NM_M,NM_M_diff_up,PE,PE_diff_up,PHIND,PHIND_diff_up,RELPOS,RELPOS_diff_up,Well Name
1383,-1.6,0.0,3007.0,8,C LM,25.16,0.00,1.112,0.000,2,0,4.8,0.0,2.890,0.000,0.030,-0.015,SHANKLE
1384,-0.4,1.2,3007.5,4,C LM,26.22,1.06,1.092,-0.020,2,0,4.5,-0.3,3.400,0.510,0.030,0.000,SHANKLE
1385,1.6,2.0,3008.0,4,C LM,65.36,39.14,1.026,-0.066,2,0,4.5,0.0,4.715,1.315,0.015,-0.015,SHANKLE


In [84]:
new_data_all23.tail(3)

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,...,PE_diff_up,NM_M_diff_up,RELPOS_diff_up,GR_diff_down,ILD_log10_diff_down,DeltaPHI_diff_down,PHIND_diff_down,PE_diff_down,NM_M_diff_down,RELPOS_diff_down
1382,8,C LM,SHANKLE,3006.5,25.16,1.112,-1.6,2.89,4.8,2,...,-0.3,0,-0.015,0.00,0.000,0.0,0.000,0.0,0,0.015
1383,8,C LM,SHANKLE,3007.0,25.16,1.112,-1.6,2.89,4.8,2,...,0.0,0,-0.015,-1.06,0.020,-1.2,-0.510,0.3,0,0.000
1384,4,C LM,SHANKLE,3007.5,26.22,1.092,-0.4,3.40,4.5,2,...,-0.3,0,0.000,-39.14,0.066,-2.0,-1.315,0.0,0,0.015


In [91]:
import numpy.random as random
test = dict()
train = dict()
test_23 = dict()
train_23 = dict()
classes = dict()
for class1 in range(9):
    for class2 in range (9):
        classes[class1, class2] = 0
acc = 0
wells = set(data['Well Name'])
wells_23 = set(new_data_23['Well Name'])
change = 0
for well in wells:
# well = 'SHRIMPLIN'
    print(well)
    test[well] = new_data[new_data['Well Name'] == well]
    test_23[well] = new_data_all23[new_data_all23['Well Name'] == well]
    train[well] = new_data[new_data['Well Name'] != well]
    X_train = train[well][feature_names].values 
    y_train = train[well]['Facies'].values 
    X_test = test[well][feature_names].values
    X_test = X_test[0:-1]
    X_test_all_23 = test_23[well][feature_names_23].values
    y_test = test[well]['Facies'].values 
    y_test = y_test[0:-1]
    well_train = train[well]['Well Name'].values
    well_test = test[well]['Well Name'].values
    well_test = well_test[0:-1]
    well_test_23 = test_23[well]['Well Name'].values
    depth_train = train[well]['Depth'].values
    depth_test = test[well]['Depth'].values   
    depth_test = depth_test[0:-1]
    depth_test_23 = test_23[well]['Depth'].values   

    X_aug_train = augment_features(X_train,well_train,depth_train)
    X_aug_test = augment_features(X_test,well_test,depth_test)
    X_aug_test_all_23 = augment_features(X_test_all_23, well_test_23, depth_test_23)

    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug_train)
    X_train_robust = robust.transform(X_aug_train)
    X_test_robust = robust.transform(X_aug_test)
    
    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug_test_all_23)
    X_test_robust_all_23 = robust.transform(X_aug_test_all_23)

    scaler = StandardScaler().fit(X_train_robust)
    X_train_robust_norm = scaler.transform(X_train_robust)
    X_test_robust_norm = scaler.transform(X_test_robust)
#     print(X_test_robust_norm[1][1:8])
    scaler = StandardScaler().fit(X_test_robust_all_23)
    X_test_robust_norm_all_23 = scaler.transform(X_test_robust_all_23)
#     print(X_test_robust_norm_all_23[1][-8:-1])
    
    dtrain = xgb.DMatrix(X_train_robust_norm, label=y_train)
    dtest = xgb.DMatrix(X_test_robust_norm, label=y_test)
    watchlist = [(dtest, 'eval'), (dtrain, 'train')]
    model = xgb.Booster(params, [dtrain])
    for _ in range(150):
        pred = model.predict(dtrain)
        g, h = my_softmax(pred, dtrain)
        model.boost(dtrain, g, h)
    yhat = model.predict(dtest)
    yhat_labels = np.argmax(yhat, axis=1)
    print("BEFORE ", f1_score(y_test, yhat_labels, average='micro'))
    if (well in wells_23):
        train_23[well] = new_data_23[new_data_23['Well Name'] != well]
        X_train_23 = train_23[well][feature_names_23].values 
        y_train_23 = train_23[well]['Facies'].values 
        well_train_23 = train_23[well]['Well Name'].values
        depth_train_23 = train_23[well]['Depth'].values
        
        X_aug_train_23 = augment_features(X_train_23,well_train_23,depth_train_23)

        robust_23 = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug_train_23)
        X_train_robust_23 = robust_23.transform(X_aug_train_23)

        scaler_23 = StandardScaler().fit(X_train_robust_23)
        X_train_robust_norm_23 = scaler_23.transform(X_train_robust_23)
        
        dtrain_23 = xgb.DMatrix(X_train_robust_norm_23, label=y_train_23)
        
        print(y_test_23)
        X_test_23 = []
        y_test_23 = []
        indeces = []
        yhat_labels_prev = []
        for index in range(len(yhat_labels)):
            if ((yhat_labels[index] == 2) or (yhat_labels[index] == 3)):
                indeces.append(index)
                X_test_23.append(X_test_robust_norm_all_23[index])
                y_test_23.append(y_test[index])
                yhat_labels_prev.append(yhat_labels[index] - 2)
        for index in range(len(y_test_23)):
            if (y_test_23[index] == 2):
                y_test_23[index] = 0
            if (y_test_23[index] == 3):
                y_test_23[index] = 1
        dtest_23 = xgb.DMatrix(X_test_23, label=y_test_23)
        watchlist = [(dtest_23, 'eval'), (dtrain_23, 'train')]
        model_23 = xgb.Booster(params_23, [dtrain_23])
        for _ in range(150):
            pred = model_23.predict(dtrain_23)
            g, h = my_softmax_23(pred, dtrain_23)
            model_23.boost(dtrain_23, g, h)
    
        yhat_23 = model_23.predict(dtest_23)
        yhat_labels_23 = np.argmax(yhat_23, axis=1)
        print("Acc in before ", f1_score(y_test_23, yhat_labels_prev, average='micro'))
        print("Acc in after ", f1_score(y_test_23, yhat_labels_23, average='micro'))
        for index in range(len(yhat_labels_23)):
            if (yhat_labels_23[index] == 0):
                yhat_labels_23[index] = 2
            if (yhat_labels_23[index] == 1):
                yhat_labels_23[index] = 3
        ind = 0
#         print("Acc in ", f1_score(y_test_23, yhat_labels_23, average='micro'))
        for index in indeces:
            if ((yhat_labels[index] == 2) and (yhat_labels_23[ind] == 3)):
                change += 1
            if ((yhat_labels[index] == 3) and (yhat_labels_23[ind] == 2)):
                change += 1
            yhat_labels[index] = yhat_labels_23[ind]
            ind  += 1
    yhat_train = model.predict(dtrain)
    yhat_labels_train = np.argmax(yhat_train, axis=1)
    print(len(y_test), " ", len(yhat_labels))
    acc += f1_score(y_test, yhat_labels, average='micro')
    print("AFTER ", f1_score(y_test, yhat_labels, average='micro'))
    print("Score on train ", f1_score(y_train, yhat_labels_train , average='micro'))
    print("Outliers test", number_of_outlier(y_test))
    print("Outliers res", number_of_outlier(yhat_labels))
    print("Change ", change)
    most_similar(yhat_labels, y_test)
print('well, boosting of trees, ', acc/10)  
print(change)

NOLAN
BEFORE  0.5423728813559322
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 8]
Acc in before  0.7447916666666666
Acc in after  0.6145833333333334
413   413
AFTER  0.48184019370460046
Score on train  0.8775838926174496
Outliers test 0.05084745762711865
Outliers res 0.053268765133171914
Change  70
LUKE G U
BEFORE  0.6405228758169934
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [92]:
data = pd.read_csv('facies_vectors_0.csv')
feature_names = ['GR_diff_up', 'ILD_log10_diff_up', 'DeltaPHI_diff_up', 'PHIND_diff_up', 'PE_diff_up', 'NM_M_diff_up', 'RELPOS_diff_up','GR_diff_down', 'ILD_log10_diff_down', 'DeltaPHI_diff_down', 'PHIND_diff_down', 'PE_diff_down', 'NM_M_diff_down', 'RELPOS_diff_down','GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']
data = data.fillna(data['PE'].mean())

In [93]:
def find_diff(row, well):
    if len(prev_depth_features[well]) == 0:
        prev_depth_features[well] = row.values[4:]
        return
    diff = row.values[4:] - prev_depth_features[well]
    prev_depth_features[well] = row.values[4:]
    return diff
data_well = dict()
data_well_inverse = dict()
prev_depth_features = dict()
new_data = pd.DataFrame()
prev_class= dict()
data_save = pd.DataFrame()
for well in set(data['Well Name']):
    prev_depth_features[well] = []
    prev_class[well] = []
    data_well[well] = data[data['Well Name'] == well]
    data_well[well] = data_well[well].sort_values(by=['Depth'])
    data_save = data_well[well].iloc[::-1]
    data_well[well]['diff_up'] = data_well[well].apply(lambda row: find_diff(row, well), axis=1)
    prev_depth_features[well] = []
    prev_class[well] = []
    data_save = data_save.apply(lambda row: find_diff(row, well), axis=1)
    data_well[well]['diff_down'] = data_save.iloc[::-1]
    data_well[well] = data_well[well].dropna()
    data_well[well]['GR_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][0], axis=1)
    data_well[well]['ILD_log10_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][1], axis=1)
    data_well[well]['DeltaPHI_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][2], axis=1)
    data_well[well]['PHIND_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][3], axis=1)
    data_well[well]['PE_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][4], axis=1)
    data_well[well]['NM_M_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][5], axis=1)
    data_well[well]['RELPOS_diff_up'] = data_well[well].apply(lambda row: row['diff_up'][6], axis=1)
    data_well[well]['GR_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][0], axis=1)
    data_well[well]['ILD_log10_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][1], axis=1)
    data_well[well]['DeltaPHI_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][2], axis=1)
    data_well[well]['PHIND_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][3], axis=1)
    data_well[well]['PE_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][4], axis=1)
    data_well[well]['NM_M_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][5], axis=1)
    data_well[well]['RELPOS_diff_down'] = data_well[well].apply(lambda row: row['diff_down'][6], axis=1)
    new_data = pd.concat([new_data, data_well[well]])
    new_data = new_data.drop(['diff_up'], axis=1)
    new_data = new_data.drop(['diff_down'], axis=1)

In [94]:
def change_23(x):
    if (x == 2):
        return 0
    if (x == 3):
        return 1
    return x
new_data_2 = new_data[new_data['Facies'] == 2]
new_data_3 = new_data[new_data['Facies'] == 3]
new_data_23 = pd.concat([new_data_2, new_data_3])
new_data_23['Facies'] = new_data_23.apply(lambda row: change_23(row['Facies']), axis = 1)

In [95]:
def augment_features_window(X, N_neig):
    N_row = X.shape[0]
    N_feat = X.shape[1]
    X = np.vstack((np.zeros((N_neig, N_feat)),np.zeros((N_neig, N_feat)), X, np.zeros((N_neig, N_feat)),np.zeros((N_neig, N_feat))))
    X_aug = np.zeros((N_row, N_feat*(4*N_neig+1)))
    for r in np.arange(N_row) + N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
            if c != 0:
                this_row = np.hstack((this_row, (X[r] + X[r+c])/2))
        #print(len(this_row))
        X_aug[r-N_neig] = this_row

    return X_aug

def augment_features_gradient(X, depth):
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad

def augment_features(X, well, depth, N_neig=1):
    X_aug = np.zeros((X.shape[0], X.shape[1]*(4*N_neig+1)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        #print(X_aug_win)
        #X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        #print(X_aug_grad)
        X_aug[w_idx, :] = X_aug_win
        #X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
        
    return X_aug

In [96]:
import numpy.random as random
test = dict()
train = dict()
acc = 0
wells = set(new_data_23['Well Name'])
for well in wells:
# well = 'SHRIMPLIN'
    print(well)
    test[well] = new_data_23[new_data_23['Well Name'] == well]
    X_test = test[well][feature_names].values 
    y_test = test[well]['Facies'].values 
    well_test = test[well]['Well Name'].values
    depth_test = test[well]['Depth'].values    

    X_aug_test = augment_features(X_test,well_test,depth_test)

    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug_test)
    X_test_robust = robust.transform(X_aug_test)

    scaler = StandardScaler().fit(X_test_robust)
    X_test_robust_norm = scaler.transform(X_test_robust)
    
    dtest = xgb.DMatrix(X_test_robust_norm, label=y_test)
    watchlist = [(dtest, 'eval'), (dtrain, 'train')]
    yhat = model_23.predict(dtest)
    yhat_labels = np.argmax(yhat, axis=1)

    acc += f1_score(y_test, yhat_labels, average='micro')
    print(f1_score(y_test, yhat_labels, average='micro'))
    print("Outliers test", number_of_outlier(y_test))
    print("Outliers res", number_of_outlier(yhat_labels))
print('well, boosting of trees, ', acc/9)  

LUKE G U
0.8530612244897959
Outliers test 0.0
Outliers res 0.04081632653061224
NEWBY
0.8079096045197741
Outliers test 0.0
Outliers res 0.022598870056497175
CHURCHMAN BIBLE
0.839622641509434
Outliers test 0.0
Outliers res 0.0660377358490566
SHRIMPLIN
0.8666666666666667
Outliers test 0.0
Outliers res 0.025
KIMZEY A
0.8490566037735849
Outliers test 0.0
Outliers res 0.018867924528301886
ALEXANDER D
0.8695652173913043
Outliers test 0.0
Outliers res 0.043478260869565216
SHANKLE
0.6536585365853659
Outliers test 0.0
Outliers res 0.07317073170731707
CROSS H CATTLE
0.8031914893617021
Outliers test 0.0
Outliers res 0.05319148936170213
NOLAN
0.8810810810810811
Outliers test 0.0
Outliers res 0.02702702702702703
well, boosting of trees,  0.8248681183754122
